In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os, datetime
import matplotlib.pyplot as plt
import numpy as np
!pip install -q tensorflow==2.0.0-beta1
import tensorflow as tf
%load_ext tensorboard

In [2]:
train_dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"
train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url),
                                           origin=train_dataset_url)

column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

feature_names = column_names[:-1]
label_name = column_names[-1]


class_names = ['Iris setosa', 'Iris versicolor', 'Iris virginica']
batch_size = 32

train_dataset = tf.data.experimental.make_csv_dataset(
    train_dataset_fp,
    batch_size,
    column_names=column_names,
    label_name=label_name,
    num_epochs=1)

features, labels = next(iter(train_dataset))

def pack_features_vector(features, labels):
  """Pack the features into a single array."""
  features = tf.stack(list(features.values()), axis=1)
  return features, labels
train_dataset = train_dataset.map(pack_features_vector)
features, labels = next(iter(train_dataset))
#print(features[:5])
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(4,)),  # input shape required
  tf.keras.layers.Dense(10, activation=tf.nn.relu),
  tf.keras.layers.Dense(3)
])
predictions = model(features)
#predictions[:5]
tf.nn.softmax(predictions[:5])
print("Prediction: {}".format(tf.argmax(predictions, axis=1)))
print("    Labels: {}".format(labels))

def plot_bar(errors, num):
    plt.hist(errors, bins = 20)
    plt.xlabel("Prediction Error [MPG]")
    title = "epoch:" + str(num)
    plt.title(title)
    _ = plt.ylabel("Count")
    plt.show()
    plt.savefig(title)

W0703 18:13:23.455134 140460800649088 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/data/experimental/ops/readers.py:498: parallel_interleave (from tensorflow.python.data.experimental.ops.interleave_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


Prediction: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
    Labels: [0 2 2 2 1 0 0 2 1 0 1 0 1 0 0 0 1 1 0 1 2 1 0 2 0 0 2 1 0 2 0 0]


## Performence without our loss function

In [0]:
from tensorflow.python.util.tf_export import keras_export
from tensorflow.python.keras.utils import losses_utils
from tensorflow.python.keras.utils import tf_utils
from tensorflow.python.keras import backend as K
import abc

@keras_export('keras.losses.Loss')
class Loss(object):

  def __init__(self, reduction=losses_utils.ReductionV2.AUTO, name=None):
    losses_utils.ReductionV2.validate(reduction)
    self.reduction = reduction
    self.name = name

  def __call__(self, y_true, y_pred, sample_weight=None):

    scope_name = 'lambda' if self.name == '<lambda>' else self.name
    graph_ctx = tf_utils.graph_context_for_symbolic_tensors(
        y_true, y_pred, sample_weight)
    with K.name_scope(scope_name or self.__class__.__name__), graph_ctx:
      losses = self.call(y_true, y_pred)
      return losses_utils.compute_weighted_loss(
          losses, sample_weight, reduction=self._get_reduction())
    
  @classmethod
  def from_config(cls, config):
    return cls(**config)

  def get_config(self):
    return {'reduction': self.reduction, 'name': self.name}

  @abc.abstractmethod
  @doc_controls.for_subclass_implementers
  def call(self, y_true, y_pred):
    NotImplementedError('Must be implemented in subclasses.')

  def _get_reduction(self):
    if self.reduction == losses_utils.ReductionV2.AUTO:
      return losses_utils.ReductionV2.SUM_OVER_BATCH_SIZE
    return self.reduction

class LossFunctionWrapper(Loss):

  def __init__(self,
               fn,
               reduction=losses_utils.ReductionV2.NONE,
               name=None,
               **kwargs):
    super(LossFunctionWrapper, self).__init__(reduction=reduction, name=name)
    self.fn = fn
    self._fn_kwargs = kwargs

  def call(self, y_true, y_pred):
    return self.fn(y_true, y_pred, **self._fn_kwargs)

  def get_config(self):
    config = {}
    for k, v in six.iteritems(self._fn_kwargs):
      config[k] = K.eval(v) if tf_utils.is_tensor_or_variable(v) else v
    base_config = super(LossFunctionWrapper, self).get_config()
    return dict(list(base_config.items()) + list(config.items()))


class SparseCategoricalCrossentropy(LossFunctionWrapper):
  def __init__(self,
               from_logits=False,
               reduction=losses_utils.ReductionV2.NONE,
               name=None):
    super(SparseCategoricalCrossentropy, self).__init__(
        sparse_categorical_crossentropy,
        name=name,
        reduction=reduction,
        from_logits=from_logits)
    
def sparse_categorical_crossentropy(y_true, y_pred, from_logits=False, axis=-1):
  return K.sparse_categorical_crossentropy(
      y_true, y_pred, from_logits=from_logits, axis=axis)
    
x = SparseCategoricalCrossentropy()

In [0]:
def loss(model, x, y):
  y_ = model(x)
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  batch_loss = loss_object(y_true=y, y_pred=y_)
  loss_object = SparseCategoricalCrossentropy(from_logits=True)
  losses = loss_object(y_true=y, y_pred=y_)
  single_losses_list = [loss.numpy() for loss in losses]
  
  return tf.convert_to_tensor(batch_loss, dtype=np.float32), single_losses_list

def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    batch_loss, single_losses_list = loss(model, inputs, targets)
  grads = tape.gradient(batch_loss, model.trainable_variables)
  return grads, batch_loss, single_losses_list


log_dir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
print("logs file name:", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
summary_writer = tf.summary.create_file_writer(logdir=log_dir)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

num_epochs = 500
train_loss_results = []
train_accuracy_results = []

losses_ce_sample = []
losses_ce_batch = []

for epoch in range(num_epochs):
  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
 
  for x, y in train_dataset:
    grads, loss_value, single_losses_list = grad(model, x, y)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    epoch_loss_avg(loss_value)
    epoch_accuracy(y, model(x))
    losses_ce_sample.append(single_losses_list)
    losses_ce_batch.append(loss_value)

  train_loss_results.append(epoch_loss_avg.result()) 
  train_accuracy_results.append(epoch_accuracy.result())

  with summary_writer.as_default():
    tf.summary.scalar('epoch_loss_avg', epoch_loss_avg.result(), step=optimizer.iterations)
    tf.summary.scalar('epoch_accuracy', epoch_accuracy.result(), step=optimizer.iterations)
    
print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,epoch_loss_avg.result(), epoch_accuracy.result()))
losses_sample = [x for sublist in losses_ce_sample for x in sublist]
losses_sample = [format(x, 'f') for x in losses_sample]
print("len(losses_sample):", len(losses_sample))
with open('losses_ce_sample.txt', 'w') as f:
    for item in losses_sample:
        f.write("%s\n" % item)
        
        
losses_ce_batch = [format(x.numpy(), 'f') for x in losses_ce_batch]
print("len(losses_ce_batch):", len(losses_batch))
with open('losses_ce_batch.txt', 'w') as f:
    for item in losses_ce_batch:
        f.write("%s\n" % item)

file name: 20190703-151603
Epoch 499: Loss: 0.032, Accuracy: 99.167%


'\nlosses_sample = trunc_list(ce_single_losses_all)\nprint("len(losses_sample):", len(losses_sample))\nwith open(\'losses_ce_sample.txt\', \'w\') as f:\n    for item in losses_sample:\n        f.write("%s\n" % item)\n        \n        \nlosses_batch = [x.numpy() for x in losses_batch]\nprint("len(losses_batch):", len(losses_batch))\nwith open(\'losses_ce_batch.txt\', \'w\') as f:\n    for item in losses_batch:\n        f.write("%s\n" % item)\n        \n'

In [5]:

print("len(losses_cees_sample):", len(losses_cees_sample))
with open('losses_cees_sample.txt', 'w') as f:
    for item in losses_cees_sample:
        f.write("%s\n" % item)
        

print("len(losses_cees_batch):", len(losses_cees_batch))
with open('losses_cees_batch.txt', 'w') as f:
    for item in losses_cees_batch:
        f.write("%s\n" % item)

len(losses_cees_sample): 480000
len(losses_cees_batch): 2000


## Performence with our loss function

In [29]:
def MMD1d(x1, x2):
  dis = sum([x**2 for x in x1])/len(x1) - sum([x**2 for x in x2])/len(x2)
  return dis**2

def get_MMD_norm(errors, sigma=0.1): 
  x2 = np.random.normal(0, sigma, 100)
  loss = MMD1d(errors, x2)
  return loss

def loss(model, x, y, sigma=0.1):
  y_ = model(x) 
  losses = []
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  for i in range(y.shape[0]):
    loss = loss_object(y_true=y[i], y_pred=y_[i])
    losses.append(loss) 
  batch_loss = get_MMD_norm(losses)
  single_losses_list = [loss.numpy() for loss in losses]
  return tf.convert_to_tensor(batch_loss, dtype=np.float32), single_losses_list

def grad(model, inputs, targets, sigma=0.1):
  with tf.GradientTape() as tape:
    tape.watch(model.trainable_variables)
    batch_loss, single_losses = loss(model, inputs, targets, sigma=0.1)
  return tape.gradient(batch_loss, model.trainable_variables), batch_loss, single_losses 


log_dir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
print("logs file name:", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
summary_writer = tf.summary.create_file_writer(logdir=log_dir)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

num_epochs = 500
train_loss_results = []
train_accuracy_results = []

losses_cees_sample = []
losses_cees_batch = []
for epoch in range(num_epochs):
  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

  for x, y in train_dataset:
    # Optimize the model
    grads, batch_loss, single_losses = grad(model, x, y)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    epoch_loss_avg(batch_loss)
    epoch_accuracy(y, model(x))
    losses_cees_sample.append(single_losses)
    losses_cees_batch.append(batch_loss)
    
  tf.summary.trace_on(graph=True, profiler=True)  
  with summary_writer.as_default():
    tf.summary.scalar('epoch_loss_avg', epoch_loss_avg.result(), step=optimizer.iterations)
    tf.summary.scalar('epoch_accuracy', epoch_accuracy.result(), step=optimizer.iterations)
    
    tf.summary.trace_export(name="my_func_trace",step=0,profiler_outdir=log_dir)

  # end epoch
  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())
  

print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,epoch_loss_avg.result(), epoch_accuracy.result()))
losses_cees_sample = [x for sublist in losses_cees_sample for x in sublist]
losses_cees_sample = [format(x, 'f') for x in losses_cees_sample]
print("len(losses_cees_sample):", len(losses_cees_sample))
with open('losses_cees_sample.txt', 'w') as f:
    for item in losses_cees_sample:
      f.write("%s\n" % item)
        
        
losses_cees_batch = [format(x.numpy(), 'f') for x in losses_cees_batch]
print("len(losses_cees_batch):", len(losses_cees_batch))
with open('losses_cees_batch.txt', 'w') as f:
    for item in losses_cees_batch:
        f.write("%s\n" % item)

logs file name: 20190703-182541
Epoch 499: Loss: 0.002, Accuracy: 98.333%
len(losses_cees_sample): 60000
len(losses_cees_batch): 2000


## The performence with our loss function and shrinking variance

In [31]:
log_dir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
print(datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
summary_writer = tf.summary.create_file_writer(logdir=log_dir)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

patience_n = 25
patience = patience_n
sigma=0.1
num_epochs = 500

train_loss_results = []
train_accuracy_results = []

losses_ceessv_sample = []
losses_ceessv_batch = []
for epoch in range(num_epochs):
  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
  if patience <= 0: 
    sigma/=2
    patience = patience_n
  # Training loop - using batches of 32
  for x, y in train_dataset:
    grads, batch_loss, single_losses = grad(model, x, y, sigma)
    losses_ceessv_sample.append(single_losses)
    losses_ceessv_batch.append(batch_loss)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Track progress
    epoch_loss_avg(batch_loss)  # add current batch loss
    # compare predicted label to actual label
    epoch_accuracy(y, model(x))
    
  tf.summary.trace_on(graph=True, profiler=True)
  # end epoch
  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())
  
  with summary_writer.as_default():
    tf.summary.scalar('epoch_loss_avg', epoch_loss_avg.result(), step=optimizer.iterations)
    tf.summary.scalar('epoch_accuracy', epoch_accuracy.result(), step=optimizer.iterations)
    
    tf.summary.trace_export(name="my_func_trace",step=0,profiler_outdir=log_dir)
      
      
  
  if epoch>1 and train_loss_results[-1].numpy() > train_loss_results[-2].numpy():
    patience -=1
    
    
print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,epoch_loss_avg.result(), epoch_accuracy.result()))
losses_ceessv_sample = [x for sublist in losses_ceessv_sample for x in sublist]
losses_ceessv_sample = [format(x, 'f') for x in losses_ceessv_sample]
print("len(losses_ceessv_sample):", len(losses_ceessv_sample))
with open('losses_ceessv_sample.txt', 'w') as f:
    for item in losses_ceessv_sample:
        f.write("%s\n" % item)
        
        
losses_ceessv_batch = [format(x.numpy(), 'f') for x in losses_ceessv_batch]
print("len(losses_ceessv_batch):", len(losses_ceessv_batch))
with open('losses_ceessv_batch.txt', 'w') as f:
    for item in losses_ceessv_batch:
        f.write("%s\n" % item)

20190703-183231
Epoch 499: Loss: 0.000, Accuracy: 99.167%
len(losses_ceessv_sample): 60000
len(losses_ceessv_batch): 2000
